In [1]:
import os
import sys
import torch
import matplotlib.pyplot as plt
import numpy as np
plt.rcParams['figure.figsize'] = [14, 8]
import motmetrics as mm
from motmetrics.distances import iou_matrix
import pandas as pd

sys.path.append('../..')
from pytracking.utils.load_text import load_text
from pytracking.evaluation import Tracker, get_dataset, trackerlist
from pprint import pprint

trackers = []
# dataset = get_dataset('probio')
# dataset = get_dataset('yt2019_sub')
dataset = get_dataset('yt_pb_valid')

def motMetricsEnhancedCalculator(gt, t):
  # import required packages
  
  acc = mm.MOTAccumulator(auto_id=True)

  # Max frame number maybe different for gt and t files
  for frame in range(int(gt[:,0].max())):
    frame += 1 # detection and frame numbers begin at 1

    # select id, x, y, width, height for current frame
    # required format for distance calculation is X, Y, Width, Height \
    # We already have this format
    gt_dets = gt[gt[:,0]==frame,1:6] # select all detections in gt
    t_dets = t[t[:,0]==frame,1:6] # select all detections in t

    C = mm.distances.iou_matrix(gt_dets[:,1:], t_dets[:,1:], \
                                max_iou=0.5) # format: gt, t

    # Call update once for per frame.
    # format: gt object ids, t object ids, distance
    acc.update(gt_dets[:,0].astype('int').tolist(), \
              t_dets[:,0].astype('int').tolist(), C)
    
  return acc

def one_track(trk):
    names = []
    accs = []

    for seq_id, seq in enumerate(dataset):
        base_results_path = '{}/{}'.format(trk.results_dir, seq.name)
        gt = []
        pred = []
        for (obj_id, obj_gt) in list(seq.ground_truth_rect.items()):
            results_path = '{}_{}.txt'.format(base_results_path,obj_id)
            if os.path.isfile(results_path):
                pred_bb = torch.tensor(load_text(str(results_path), delimiter=('\t', ','), dtype=np.float64))
                obj_gt = torch.tensor(obj_gt,dtype=torch.float64)
            else:
                print('Result not found. {}'.format(results_path))
                continue
            for frame_id, b in enumerate(obj_gt):
                if b[0] == -1: continue
                gt.append([frame_id+1,obj_id,b[0],b[1],b[2],b[3],1.,-1.,-1.,-1.])
            for frame_id, b in enumerate(pred_bb):
                if b[0] == -1: continue
                pred.append([frame_id+1,obj_id,b[0],b[1],b[2],b[3],1.,-1.,-1.,-1.])
        # if seq.name in ['0043f083b5','0044fa5fba']:
        #     slices = slice(2,6)
        #     print(f'----------------{seq.name}-------------')
        #     cur_id = 0
        #     for i,j in zip(gt,pred):
        #         if i[1] != cur_id:
        #             print(f'----------------{i[1]}-------------')
        #             cur_id = i[1]
        #         a = [f'{int(ii)-int(jj):>3}' for ii,jj in zip(i[slices],j[slices])]
        #         bi = [int(ii) for ii in i[slices]]
        #         bj = [int(jj) for jj in j[slices]]
        #         c = iou_matrix([i[slices]],[j[slices]])[0][0]
        #         print(f'{c:>5.3f}',a,bi,bj)
        if len(gt) == 0: continue
        accs.append(motMetricsEnhancedCalculator(np.array(gt, dtype=np.float64), np.array(pred, dtype=np.float64)))
        names.append(seq.name)


    mh = mm.metrics.create()
    summary = mh.compute_many(accs, metrics=['mota', 'motp', 'num_frames', 'num_objects' , 'idf1', 'idp', 'idr', \
                                        'recall', 'precision',  \
                                        'mostly_tracked', 'partially_tracked', \
                                        'mostly_lost', 'num_false_positives', \
                                        'num_misses', 'num_switches', \
                                        'num_fragmentations'
                                    ], \
                        generate_overall=True,
                        names=names)

    # strsummary = mm.io.render_summary(
    #     summary,
    #     #formatters={'mota' : '{:.2%}'.format},
    #     namemap={'num_objects': 'GT', 'mota': 'MOTA', 'motp' : 'MOTP', \
    #              'idf1': 'IDF1', 'idp': 'IDP', 'idr': 'IDR', 'recall': 'Rcll', \
    #             'precision': 'Prcn',  \
    #             'mostly_tracked' : 'MT', 'partially_tracked': 'PT', \
    #             'mostly_lost' : 'ML', 'num_false_positives': 'FP', \
    #             'num_misses': 'FN', 'num_switches' : 'IDsw', \
    #             'num_fragmentations' : 'FM'
    #             }
    # )

    df =  pd.DataFrame.from_dict(summary)
    namemap = {'num_objects': 'GT', 'mota': 'MOTA', 'motp' : 'MOTP', \
                 'idf1': 'IDF1', 'idp': 'IDP', 'idr': 'IDR', 'recall': 'Rcll', 'precision': 'Prcn',  \
                'mostly_tracked' : 'MT', 'partially_tracked': 'PT', \
                'mostly_lost' : 'ML', 'num_false_positives': 'FP', \
                'num_misses': 'FN', 'num_switches' : 'IDsw', \
                'num_fragmentations' : 'FM', 'num_frames': 'Frames'
                }
    df.rename(columns=namemap, inplace=True)
    return df

# tkl = []
# pt = '../tracking_results'
# for trk in os.listdir(pt):
#     for param in os.listdir(os.path.join(pt, trk)):
#         tkl.append([trk, param, trackerlist(trk, param, None)[0]])
# pprint(tkl,compact=True,width=120)

YouTubeVOS/2019/probio_valid loaded.


In [48]:
# for i in range(len(tkl)):
#     result = one_track(tkl[i][-1])
#     trk,param = tkl[i][0], tkl[i][1]
#     result.to_csv(os.path.join('../tracking_csv', 'result_{}_{}.csv'.format(trk, param)), index=False)

def sam_track():
    names = []
    accs = []

    for seq_id, seq in enumerate(dataset):
        gt = []
        pred = []
        for (obj_id, obj_gt) in list(seq.ground_truth_rect.items()):
            for frame_id, b in enumerate(obj_gt):
                if b[0] == -1: continue
                gt.append([frame_id+1,255-int(obj_id),b[0],b[1],b[2],b[3],1.,-1.,-1.,-1.])
        pred_base_results = '/mnt/data/qizhezhang/TSAMs/segment-anything-2/results'
        with open(os.path.join(pred_base_results, seq.name+'.txt')) as f:
            lines = f.readlines() 
            pred.append([int(i) for i in lines[0].split(',')])
        accs.append(motMetricsEnhancedCalculator(np.array(gt, dtype=np.float64), np.array(pred, dtype=np.float64)))
        names.append(seq.name)


    mh = mm.metrics.create()
    summary = mh.compute_many(accs, metrics=['mota', 'motp', 'num_frames', 'num_objects' , 'idf1', 'idp', 'idr', \
                                        'recall', 'precision',  \
                                        'mostly_tracked', 'partially_tracked', \
                                        'mostly_lost', 'num_false_positives', \
                                        'num_misses', 'num_switches', \
                                        'num_fragmentations'
                                    ], \
                        generate_overall=True,
                        names=names)

    df =  pd.DataFrame.from_dict(summary)
    namemap = {'num_objects': 'GT', 'mota': 'MOTA', 'motp' : 'MOTP', \
                 'idf1': 'IDF1', 'idp': 'IDP', 'idr': 'IDR', 'recall': 'Rcll', 'precision': 'Prcn',  \
                'mostly_tracked' : 'MT', 'partially_tracked': 'PT', \
                'mostly_lost' : 'ML', 'num_false_positives': 'FP', \
                'num_misses': 'FN', 'num_switches' : 'IDsw', \
                'num_fragmentations' : 'FM', 'num_frames': 'Frames'
                }
    df.rename(columns=namemap, inplace=True)
    return df

sam_track()

,MOTA,MOTP,Frames,GT,IDF1,IDP,IDR,Rcll,Prcn,MT,PT,ML,FP,FN,IDsw,FM
004,0.002347,0.086957,39,426,0.004684,1.000000,0.002347,0.002347,1.000000,0,0,11,0,425,0,0
007,0.008850,0.044038,13,113,0.017544,1.000000,0.008850,0.008850,1.000000,0,0,9,0,112,0,0
008,0.003636,0.076923,25,275,0.007246,1.000000,0.003636,0.003636,1.000000,0,0,11,0,274,0,0
010,0.005319,0.050000,18,188,0.010582,1.000000,0.005319,0.005319,1.000000,0,0,11,0,187,0,0
016,0.003125,0.101351,20,320,0.006231,1.000000,0.003125,0.003125,1.000000,0,0,16,0,319,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155,0.000793,0.049853,90,1261,0.001585,1.000000,0.000793,0.000793,1.000000,0,0,15,0,1260,0,0
158,0.001182,0.049853,55,846,0.002361,1.000000,0.001182,0.001182,1.000000,0,0,16,0,845,0,0
159,0.000592,0.049853,110,1689,0.001183,1.000000,0.000592,0.000592,1.000000,0,0,18,0,1688,0,0
163,0.002611,0.007752,43,383,0.005208,1.000000,0.002611,0.002611,1.000000,0,0,9,0,382,0,0


In [52]:
def sam_visualize():
    from collections import defaultdict
    seq = dataset[0]
    gt = defaultdict(dict)
    pred = defaultdict(dict)
    
    for (obj_id, obj_gt) in list(seq.ground_truth_rect.items()):
        for frame_id, b in enumerate(obj_gt):
            gt[frame_id][obj_id] = [int(i) for i in b]
    pred_base_results = '/mnt/data/qizhezhang/TSAMs/segment-anything-2/results'
    with open(os.path.join(pred_base_results, seq.name+'.txt')) as f:
        for line in f.readlines():
            l = line.split(',') # lines读到的信息是[frame_id+1,obj_id,b[0],b[1],b[2],b[3],1.,-1.,-1.,-1.]
            pred[int(l[0])-1][int(l[1])] = [int(i) for i in l[2:6]]
    
    def draw(i):
        img_path = seq.frames[i]
        import cv2
        img1 = cv2.imread(img_path)
        img2 = cv2.imread(img_path)
        
        for obj_id, b in gt[i].items():
            cv2.rectangle(img1, (b[0],b[1]), (b[0]+b[2],b[1]+b[3]), (0,255,0), 1)
            cv2.putText(img1, str(255-int(obj_id)), (b[0],b[1]), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 182, 193), 1)
        for obj_id, b in pred[i].items():
            cv2.rectangle(img2, (b[0],b[1]), (b[0]+b[2],b[1]+b[3]), (0,0,255), 1)
            cv2.putText(img2, str(obj_id), (b[0],b[1]), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,200,200), 1)
            
        # 把两张图片并排显示
        img = np.hstack((img1, img2))
        # save
        cv2.imwrite(f'sam_visualize/visualize_{i}.png', img)
        
    for i in range(len(gt)):
        draw(i)
            
    
sam_visualize()